<a href="https://colab.research.google.com/github/cechus/cechus.github.io/blob/master/GDG_EL_ALTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM WATSON NATURAL LANGUAGE UNDERSTANDING


In [97]:
!pip install --upgrade "ibm-watson>=4.1.0"

Requirement already up-to-date: ibm-watson>=4.1.0 in /usr/local/lib/python3.6/dist-packages (4.4.0)


In [0]:
import pandas as pd
import pickle

### Se importa el archivo pickle con los textos a clasificar

In [99]:
normas = pickle.load(open("pdf_preprocesado.pkl","rb"))
# indices = list(normas.index.values.astype("int"))
normas.head(6)

,objeto_lemma,archivo,pagina,OBJETO,NROTOPIC
0,"[legislativa, periodo, legislativo, ley, dicie...",0.pdf,0,asamblea legislativa departamental pando perio...,NaN
1,"[legislativa, periodo, legislativo, pdo, cruz,...",0.pdf,1,asamblea legislativa departamental pando perio...,NaN
2,"[legislativa, periodo, legislativo, delgado, p...",0.pdf,2,legislativa asamblea departamental pando perio...,NaN
3,"[legislativa, periodo, legislativo, nadia, pdo...",0.pdf,3,asamblea legislativa departamental pando perio...,NaN
4,"[ley, cuanto, uso, especificas, atribuciones, ...",1.pdf,0,oeparta slatiha asamбlea dерartamcntal oruro l...,NaN
5,"[gobierno, legislativa, ley, ley, agosto, ley,...",10.pdf,0,gobierno autonomo departamental cochabamba asa...,NaN


In [100]:
normas['objeto_lemma'][0]


['legislativa',
 'periodo',
 'legislativo',
 'ley',
 'diciembre',
 'adolfo',
 'cuanto',
 'legislativa',
 'sancionado',
 'siguiente',
 'ley',
 'ley',
 'enmendar',
 'ley',
 'junio',
 'enmendar',
 'primero',
 'ley',
 'junio',
 'deber',
 'quedar',
 'siguiente',
 'manera',
 'conformidad',
 'establecido',
 'paragrafo',
 'numeral',
 'estatuto',
 'dispuesto',
 'ley',
 'decreto',
 'mts',
 'distribuidos',
 'lotes',
 'terreno',
 'insertos',
 'mts',
 'zona',
 'paz',
 'propiedad',
 'gobierno',
 'registradas',
 'bajar',
 'matricula',
 'favor',
 'siguientes',
 'beneficiarios',
 'aprobar',
 'titulo',
 'superficie',
 'superficie',
 'total',
 'derechos',
 'reales',
 'superficie',
 'marcelo',
 'luisa',
 'pdo',
 'cristhian',
 'pena',
 'pdo',
 'pdo',
 'tatiana',
 'judith',
 'cardona',
 'pdo',
 'zelada',
 'pdo',
 'alvarado',
 'febrero']

In [101]:
normas['objeto_lemma'][10]


['decreto',
 'gonzales',
 'bernal',
 'constitucional',
 'potosi',
 'considerando',
 'determinar',
 'competencias',
 'exclusivas',
 'gobiernos',
 'departamentales',
 'dirigido',
 'autoridad',
 'ley',
 'mayo',
 'art',
 'disponer',
 'gobiernos',
 'departamentales',
 'estructura',
 'administrativa',
 'conformar',
 'sistemas',
 'ley',
 'julio',
 'control',
 'gubernamentales',
 'disposiciones',
 'concordantes',
 'igual',
 'forma',
 'art',
 'disponer',
 'transferencia',
 'gobiernos',
 'departamentales',
 'derechos',
 'obligaciones',
 'donaciones',
 'recursos',
 'financieros',
 'financieros',
 'patrimonio',
 'activos',
 'pasivos',
 'prefecturas',
 'departamentales',
 'conformidad',
 'establecido',
 'art',
 'ley',
 'concordante',
 'reglamento',
 'empresas',
 'operadoras',
 'turismo',
 'receptivo',
 'empresas',
 'viajes',
 'turismo',
 'empresas',
 'operadoras',
 'turismo',
 'receptivo',
 'disponer',
 'vez',
 'art',
 'facultad',
 'autorizar',
 'inscripcion',
 'registro',
 'turismo',
 'acuerdo',
 

### Autenticación para el uso del modelo de Watson

In [0]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features,CategoriesOptions,KeywordsOptions
import json

authenticator =                                                                                       IAMAuthenticator("BF6nMz8cEHwEoBqSC1LUwlmjGsDYAQ6LpvpaqRo4dQhO")
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)
natural_language_understanding.set_service_url("https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/923af3f4-8bf2-47d5-a0a8-2840d9064a22")

In [103]:
normas.shape[0]

78532

### Generación del objeto respuesta con las categorias y keywords(textos relevantes) del modelo de Watson

In [104]:
from ibm_watson import ApiException

watson75 = pd.DataFrame(columns=["output_watson"])
indice = 0
# limitNormas = normas.shape[0]
limitNormas = 100 
while(indice < limitNormas):
    
    try:
        for i in range(indice, limitNormas):
            text=normas["OBJETO"][i]
            response = natural_language_understanding.analyze(
                text=text,
                features=Features(
                categories=CategoriesOptions(limit=3),
        #             entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
                keywords=KeywordsOptions(emotion=True, sentiment=True, limit=3)),language="es").get_result()

            lista_general = []
            lista_keywords = []
            lista_categories =  []

            for keyword in response["keywords"]:
                text = keyword["text"]
                relevance = keyword["relevance"]
                lista_keywords.append(tuple((text+","+str(relevance)).split(",")))

            for category in response["categories"]:
                lista_categories.append(category)

            lista_general.append(lista_keywords)
            lista_general.append(lista_categories)
            print("\n", "nro: ", i)
            print("\n", response)

            watson75 = watson75.append({"output_watson":lista_general}, ignore_index=True)

    except ApiException as ex:
        watson75 = watson75.append({"output_watson": "error"}, ignore_index=True)
        print("\n", "nro----------------->", i)
        indice = i+1
        print("\n", "indice----------------->", indice)
        pass

    finally:
        print("The 'try except' is finished")
        indice = i + 1

# Envía un mensaje


 nro:  0

 {'usage': {'text_units': 1, 'text_characters': 1237, 'features': 2}, 'language': 'es', 'keywords': [{'text': 'ley departamental', 'sentiment': {'score': 0, 'label': 'neutral'}, 'relevance': 0.813381, 'count': 2}, {'text': 'ley ley departamental', 'sentiment': {'score': 0, 'label': 'neutral'}, 'relevance': 0.660806, 'count': 1}, {'text': 'periodo legislativo', 'sentiment': {'score': 0, 'label': 'neutral'}, 'relevance': 0.631729, 'count': 1}], 'categories': [{'score': 0.85222, 'label': '/law, govt and politics/government'}, {'score': 0.845941, 'label': '/law, govt and politics/government/legislative'}, {'score': 0.729781, 'label': '/law, govt and politics/government/parliament'}]}

 nro:  1

 {'usage': {'text_units': 1, 'text_characters': 782, 'features': 2}, 'language': 'es', 'keywords': [{'text': 'сoii pdo', 'sentiment': {'score': 0, 'label': 'neutral'}, 'relevance': 0.980721, 'count': 1}, {'text': 'caraica cruz pdo', 'sentiment': {'score': 0, 'label': 'neutral'}, 'relevanc

In [105]:
response

{'categories': [{'label': '/sports/fishing', 'score': 0.656152},
  {'label': '/sports/fishing/saltwater fishing', 'score': 0.621641},
  {'label': '/sports/fishing/freshwater fishing', 'score': 0.610763}],
 'keywords': [{'count': 1,
   'relevance': 0.71544,
   'sentiment': {'label': 'positive', 'score': 0.333131},
   'text': 'eiuzacion dap estructlrantesy'},
  {'count': 1,
   'relevance': 0.604003,
   'sentiment': {'label': 'positive', 'score': 0.333131},
   'text': 'utonomo manicipal'},
  {'count': 1,
   'relevance': 0.573549,
   'sentiment': {'label': 'positive', 'score': 0.333131},
   'text': 'msenforizacion gob wun'}],
 'language': 'es',
 'usage': {'features': 2, 'text_characters': 1008, 'text_units': 1}}

### Dataframe resultado de la asignación de los keywords y las categorias

In [106]:
watson75.shape

(100, 1)

In [107]:
watson75['output_watson'][0]

[[('ley departamental', '0.813381'),
  ('ley ley departamental', '0.660806'),
  ('periodo legislativo', '0.631729')],
 [{'label': '/law, govt and politics/government', 'score': 0.85222},
  {'label': '/law, govt and politics/government/legislative',
   'score': 0.845941},
  {'label': '/law, govt and politics/government/parliament',
   'score': 0.729781}]]

In [108]:
watson75['output_watson'][10]

[[('decreto departamental felix', '0.829535'),
  ('gobiernos departamentales', '0.717856'),
  ('autonomos jurisdiccion', '0.649457')],
 [{'label': '/law, govt and politics', 'score': 0.849727},
  {'label': '/travel/tourist facilities', 'score': 0.770399},
  {'label': '/style and fashion/body art', 'score': 0.764784}]]

In [109]:
watson75['output_watson'][53]

[[('exposicion motivos constitucion', '0.718246'),
  ('ley municipal', '0.68935'),
  ('alcalde municipal', '0.645381')],
 [{'label': '/law, govt and politics/government', 'score': 0.966185},
  {'label': '/law, govt and politics/government/parliament',
   'score': 0.866914},
  {'label': '/law, govt and politics/politics', 'score': 0.827719}]]

### Exportación del dataframe resultado a un archivo pickle final

In [0]:
watson75.to_pickle("watson_result.pkl")